In [54]:
import json
import pandas as pd
import requests
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

/home/gbemike/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [25]:
options = webdriver.ChromeOptions()

options.set_capability(
    'goog:loggingPrefs', 
    {
        'performance': 'ALL', 
        'browser': 'ALL'
        }
)

driver = webdriver.Chrome(options = options)
driver.set_page_load_timeout(20)

In [26]:
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

In [120]:
matches = pd.read_csv('matches.csv')

In [121]:
matches

,matches,match_id
0,germany-scotland,VTbslUb
1,hungary-switzerland,ZTbsjUb
2,croatia-spain,YTbspUb
3,italy-albania,PTbshUb
4,netherlands-poland,dUbsfUb
5,slovenia-denmark,BObsJOb
6,serbia-england,nUbsfCc
7,ukraine-romania,CObsbUb
8,belgium-slovakia,XTbsrUb
9,austria-france,GObstUb


In [129]:
match_list = matches['matches'].to_list()
id_lsit = matches['match_id'].to_list()

In [ ]:
standard_columns = [
'shirtNumber', 'jerseyNumber', 'position', 'substitute',
       'player.slug', 'player.position',
       'player.jerseyNumber','player.country.name',
       'statistics.totalPass', 'statistics.accuratePass',
       'statistics.totalLongBalls', 'statistics.goalAssist',
       'statistics.minutesPlayed', 'statistics.touches', 'statistics.rating',
       'statistics.possessionLostCtrl','statistics.accurateLongBalls',
       'statistics.totalCross', 'statistics.accurateCross',
       'statistics.duelLost', 'statistics.duelWon', 'statistics.challengeLost',
       'statistics.totalContest', 'statistics.blockedScoringAttempt',
       'statistics.totalClearance', 'statistics.totalTackle',
       'statistics.expectedGoals', 'statistics.keyPass',
       'statistics.expectedAssists', 'statistics.aerialLost',
       'statistics.aerialWon', 'statistics.onTargetScoringAttempt',
       'statistics.ownGoals', 'statistics.fouls', 'statistics.wasFouled',
       'statistics.totalOffside', 'statistics.shotOffTarget',
       'statistics.interceptionWon', 'statistics.dispossessed',
       'statistics.wonContest', 'statistics.goals', 'captain',
       'statistics.penaltyWon'
]

In [ ]:
match_df = []
base_url = 'https://www.sofascore.com'

for match, match_id in zip(match_list, id_list):
    match_url = f'{base_url}/{match}/{match_id}'
    try:
        driver.get(match_url)
    except:
        pass
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # gives all network responses/ all APIs being called
    logs_raw = driver.get_log('performance')

    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

    for x in logs:
    if 'lineups' in x['params'].get('headers', {}).get(':path', ''):
        print(x['params'].get('headers', {}).get(':path'))
        break

    lineups_url = x['params'].get('headers', {}).get(':path')
    r = requests.get(urljoin("https://sofascore.com", lineups_url), headers = headers)

    if r.status_code == 200:
        data = r.json()

    home = data['home']['players']

    home_lineups = pd.json_normalize(home)

    # Ensure all necessary columns from standard column list are present
    home_lineups = home_lineups.reindex(columns=standard_columns)

    home_lineups.columns = home_lineups.columns.str.replace('statistics.','')
    home_lineups.columns = home_lineups.columns.str.replace('player.','')
    home_lineups.columns = home_lineups.columns.str.replace('slug','player_name')

    away = data['away']['players']

    away_lineups = pd.json_normalize(away)

    # Ensure all necessary columns from standard column list are present
    away_lineups = away_lineups.reindex(columns=standard_columns)

    away_lineups.columns = away_lineups.columns.str.replace('statistics.','')
    away_lineups.columns = away_lineups.columns.str.replace('player.','')
    away_lineups.columns = away_lineups.columns.str.replace('slug','player_name')

In [27]:
# matches = "germany-scotland"
# match_ids = "VTbslUb"


# # for match, match_id in zip(matches, match_ids):

# match_url = f'{base_url}/{matches}/{match_ids}'

# try:
#     driver.get(match_url)
# except:
#     pass

# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


In [28]:
# gives all network responses/ all APIs being called
logs_raw = driver.get_log('performance')

In [29]:
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

In [30]:
logs[0]

{'method': 'Network.loadingFinished',
 'params': {'encodedDataLength': 0,
  'requestId': '54EC0E8DFA2994E8929F718B97DC5344',
  'timestamp': 18003.839078}}

In [31]:
for x in logs:
    if 'lineups' in x['params'].get('headers', {}).get(':path', ''):
        print(x['params'].get('headers', {}).get(':path'))
        break

/api/v1/event/11873905/lineups


In [32]:
print(x['params'].get('headers', {}).get(':path'))

/api/v1/event/11873905/lineups


In [81]:
from urllib.parse import urljoin

lineups_url = x['params'].get('headers', {}).get(':path')
r = requests.get(urljoin("https://sofascore.com", lineups_url), headers = headers)

if r.status_code == 200:
    data = r.json()

In [82]:
data.keys()

dict_keys(['confirmed', 'home', 'away'])

In [83]:
home = data['home']['players']

In [84]:
home_lineups = pd.json_normalize(home)

In [85]:
home_lineups.columns

Index(['shirtNumber', 'jerseyNumber', 'position', 'substitute', 'player.name',
       'player.slug', 'player.shortName', 'player.position',
       'player.jerseyNumber', 'player.userCount', 'player.id',
       'player.country.alpha2', 'player.country.alpha3', 'player.country.name',
       'player.marketValueCurrency', 'player.dateOfBirthTimestamp',
       'player.fieldTranslations.nameTranslation.ar',
       'player.fieldTranslations.shortNameTranslation.ar',
       'statistics.totalPass', 'statistics.accuratePass',
       'statistics.totalLongBalls', 'statistics.goalAssist',
       'statistics.minutesPlayed', 'statistics.touches', 'statistics.rating',
       'statistics.possessionLostCtrl', 'statistics.ratingVersions.original',
       'statistics.ratingVersions.alternative', 'player.firstName',
       'player.lastName', 'statistics.accurateLongBalls',
       'statistics.totalCross', 'statistics.accurateCross',
       'statistics.duelLost', 'statistics.duelWon', 'statistics.challengeLo

In [86]:
standard_columns = [
'shirtNumber', 'jerseyNumber', 'position', 'substitute',
       'player.slug', 'player.position',
       'player.jerseyNumber','player.country.name',
       'statistics.totalPass', 'statistics.accuratePass',
       'statistics.totalLongBalls', 'statistics.goalAssist',
       'statistics.minutesPlayed', 'statistics.touches', 'statistics.rating',
       'statistics.possessionLostCtrl','statistics.accurateLongBalls',
       'statistics.totalCross', 'statistics.accurateCross',
       'statistics.duelLost', 'statistics.duelWon', 'statistics.challengeLost',
       'statistics.totalContest', 'statistics.blockedScoringAttempt',
       'statistics.totalClearance', 'statistics.totalTackle',
       'statistics.expectedGoals', 'statistics.keyPass',
       'statistics.expectedAssists', 'statistics.aerialLost',
       'statistics.aerialWon', 'statistics.onTargetScoringAttempt',
       'statistics.ownGoals', 'statistics.fouls', 'statistics.wasFouled',
       'statistics.totalOffside', 'statistics.shotOffTarget',
       'statistics.interceptionWon', 'statistics.dispossessed',
       'statistics.wonContest', 'statistics.goals', 'captain',
       'statistics.penaltyWon'
]

In [87]:
len(standard_columns)

44

In [88]:
# Ensure all necessary columns are present
home_lineups = home_lineups.reindex(columns=standard_columns)

In [89]:
home_lineups.columns

Index(['shirtNumber', 'jerseyNumber', 'position', 'substitute', 'player.name',
       'player.slug', 'player.position', 'player.jerseyNumber',
       'player.country.name', 'statistics.totalPass',
       'statistics.accuratePass', 'statistics.totalLongBalls',
       'statistics.goalAssist', 'statistics.minutesPlayed',
       'statistics.touches', 'statistics.rating',
       'statistics.possessionLostCtrl', 'statistics.accurateLongBalls',
       'statistics.totalCross', 'statistics.accurateCross',
       'statistics.duelLost', 'statistics.duelWon', 'statistics.challengeLost',
       'statistics.totalContest', 'statistics.blockedScoringAttempt',
       'statistics.totalClearance', 'statistics.totalTackle',
       'statistics.expectedGoals', 'statistics.keyPass',
       'statistics.expectedAssists', 'statistics.aerialLost',
       'statistics.aerialWon', 'statistics.onTargetScoringAttempt',
       'statistics.ownGoals', 'statistics.fouls', 'statistics.wasFouled',
       'statistics.totalO

In [95]:
home_lineups.columns = home_lineups.columns.str.replace('statistics.','')
home_lineups.columns = home_lineups.columns.str.replace('player.','')
home_lineups.columns = home_lineups.columns.str.replace('slug','player_name')

In [96]:
home_lineups.columns

Index(['shirtNumber', 'jerseyNumber', 'position', 'substitute', 'name', 'slug',
       'position', 'jerseyNumber', 'country.name', 'totalPass', 'accuratePass',
       'totalLongBalls', 'goalAssist', 'minutesPlayed', 'touches', 'rating',
       'possessionLostCtrl', 'accurateLongBalls', 'totalCross',
       'accurateCross', 'duelLost', 'duelWon', 'challengeLost', 'totalContest',
       'blockedScoringAttempt', 'totalClearance', 'totalTackle',
       'expectedGoals', 'keyPass', 'expectedAssists', 'aerialLost',
       'aerialWon', 'onTargetScoringAttempt', 'ownGoals', 'fouls', 'wasFouled',
       'totalOffside', 'shotOffTarget', 'interceptionWon', 'dispossessed',
       'wonContest', 'goals', 'captain', 'penaltyWon'],
      dtype='object')

In [97]:
home_lineups.shape

(26, 44)

In [99]:
home_lineups['touches']

0      15.0
1      97.0
2     102.0
3      98.0
4      89.0
5      34.0
6     108.0
7      59.0
8      45.0
9      46.0
10     25.0
11     73.0
12     24.0
13      5.0
14     12.0
15     13.0
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
Name: touches, dtype: float64

In [100]:
away = data['away']['players']

In [102]:
away_lineups = pd.json_normalize(away)

In [103]:
# Ensure all necessary columns are present
away_lineups = away_lineups.reindex(columns=standard_columns)

In [104]:
away_lineups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 44 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   shirtNumber                        26 non-null     int64  
 1   jerseyNumber                       26 non-null     object 
 2   position                           26 non-null     object 
 3   substitute                         26 non-null     bool   
 4   player.name                        26 non-null     object 
 5   player.slug                        26 non-null     object 
 6   player.position                    26 non-null     object 
 7   player.jerseyNumber                22 non-null     object 
 8   player.country.name                26 non-null     object 
 9   statistics.totalPass               16 non-null     float64
 10  statistics.accuratePass            16 non-null     float64
 11  statistics.totalLongBalls          11 non-null     float64
 

In [105]:
away_lineups.shape

(26, 44)

In [106]:
away_lineups.columns = away_lineups.columns.str.replace('statistics.','')
away_lineups.columns = away_lineups.columns.str.replace('player.','')
away_lineups.columns = away_lineups.columns.str.replace('slug','player_name')

In [107]:
away_lineups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   shirtNumber             26 non-null     int64  
 1   jerseyNumber            26 non-null     object 
 2   position                26 non-null     object 
 3   substitute              26 non-null     bool   
 4   name                    26 non-null     object 
 5   slug                    26 non-null     object 
 6   position                26 non-null     object 
 7   jerseyNumber            22 non-null     object 
 8   country.name            26 non-null     object 
 9   totalPass               16 non-null     float64
 10  accuratePass            16 non-null     float64
 11  totalLongBalls          11 non-null     float64
 12  goalAssist              16 non-null     float64
 13  minutesPlayed           16 non-null     float64
 14  touches                 16 non-null     floa

In [108]:
away_lineups.columns

Index(['shirtNumber', 'jerseyNumber', 'position', 'substitute', 'name', 'slug',
       'position', 'jerseyNumber', 'country.name', 'totalPass', 'accuratePass',
       'totalLongBalls', 'goalAssist', 'minutesPlayed', 'touches', 'rating',
       'possessionLostCtrl', 'accurateLongBalls', 'totalCross',
       'accurateCross', 'duelLost', 'duelWon', 'challengeLost', 'totalContest',
       'blockedScoringAttempt', 'totalClearance', 'totalTackle',
       'expectedGoals', 'keyPass', 'expectedAssists', 'aerialLost',
       'aerialWon', 'onTargetScoringAttempt', 'ownGoals', 'fouls', 'wasFouled',
       'totalOffside', 'shotOffTarget', 'interceptionWon', 'dispossessed',
       'wonContest', 'goals', 'captain', 'penaltyWon'],
      dtype='object')

In [109]:
away_lineups.head()

,shirtNumber,jerseyNumber,position,substitute,name,slug,position,jerseyNumber,country.name,totalPass,...,fouls,wasFouled,totalOffside,shotOffTarget,interceptionWon,dispossessed,wonContest,goals,captain,penaltyWon
0,1,1,G,False,Angus Gunn,angus-gunn,G,28,Scotland,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,D,False,Anthony Ralston,anthony-ralston,D,56,Scotland,21.0,...,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,15,D,False,Ryan Porteous,porteous-ryan,D,5,Scotland,21.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,13,D,False,Jack Hendry,jack-hendry,D,4,Scotland,26.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,6,D,False,Kieran Tierney,kieran-tierney,D,17,Scotland,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
away_lineups['rating']

0     6.5
1     5.9
2     5.1
3     5.9
4     6.1
5     6.0
6     6.0
7     6.7
8     6.3
9     6.3
10    6.4
11    6.4
12    6.3
13    6.2
14    6.8
15    6.5
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
Name: rating, dtype: float64

In [111]:
lineups = pd.concat([home_lineups, away_lineups], ignore_index=True)

In [112]:
lineups

,shirtNumber,jerseyNumber,position,substitute,name,slug,position,jerseyNumber,country.name,totalPass,...,fouls,wasFouled,totalOffside,shotOffTarget,interceptionWon,dispossessed,wonContest,goals,captain,penaltyWon
0,1,1,G,False,Manuel Neuer,manuel-neuer,G,1,Germany,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,6,D,False,Joshua Kimmich,joshua-kimmich,M,6,Germany,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,D,False,Antonio Rüdiger,antonio-rudiger,D,22,Germany,91.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,D,False,Jonathan Tah,jonathan-tah,D,4,Germany,93.0,...,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,18,D,False,Maximilian Mittelstädt,maximilian-mittelstadt,D,7,Germany,62.0,...,2.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5,23,23,M,False,Robert Andrich,robert-andrich,M,8,Germany,28.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8,8,M,False,Toni Kroos,toni-kroos,M,NaN,Germany,102.0,...,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
7,10,10,M,False,Jamal Musiala,jamal-musiala,M,42,Germany,32.0,...,NaN,2.0,NaN,NaN,1.0,3.0,5.0,1.0,NaN,NaN
8,21,21,M,False,İlkay Gündoğan,ilkay-gundogan,M,22,Germany,34.0,...,1.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,True,1.0
9,17,17,M,False,Florian Wirtz,florian-wirtz,F,10,Germany,31.0,...,NaN,1.0,1.0,2.0,NaN,1.0,1.0,1.0,NaN,NaN


In [113]:
lineups.to_csv('linueps.csv', mode='a')